Clustering Task

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('processed_data.csv')

# Optimize data types to reduce memory
df = df.astype(
    {col: 'float32' for col in df.select_dtypes(include='float64').columns})

# Select features for clustering
features = ['demand', 'precipIntensity', 'precipProbability', 'temperature', 'humidity',
            'pressure', 'windSpeed', 'hour', 'day_of_week', 'month', 'year', 'anomaly']
X = df[features]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X).astype(
    np.float32)  # Use float32 to save memory

# 1. Dimensionality Reduction
# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
explained_variance = pca.explained_variance_ratio_

# t-SNE (subsample for speed)
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=250)
# Reduced subsample for faster computation
X_tsne = tsne.fit_transform(X_scaled[:5000])

# Visualize PCA and t-SNE
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.3, s=10)
plt.title(f'PCA (Explained Variance: {explained_variance.sum():.2%})')
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.subplot(1, 2, 2)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], alpha=0.3, s=10)
plt.title('t-SNE')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.tight_layout()
plt.savefig('dimensionality_reduction.png')
plt.close()

# 2. Clustering Algorithms
# K-Means: Elbow Method
inertia = []
K = range(1, 10)  # Reduced range for speed
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(6, 4))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.savefig('elbow_curve.png')
plt.close()

optimal_k = 4  # Adjust based on elbow curve
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
dbscan_labels = dbscan.fit_predict(X_scaled)

n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)
print(f'DBSCAN: {n_clusters_dbscan} clusters, {n_noise} noise points')

# Hierarchical Clustering (subsample to avoid memory error)
sample_size = 1000  # Optimized for memory
np.random.seed(42)
sample_indices = np.random.choice(
    X_scaled.shape[0], sample_size, replace=False)
X_sample = X_scaled[sample_indices]

# Dendrogram
Z = linkage(X_sample, method='ward', metric='euclidean')
plt.figure(figsize=(8, 4))
dendrogram(Z, truncate_mode='level', p=5)
plt.title('Dendrogram for Hierarchical Clustering')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.savefig('dendrogram.png')
plt.close()

# Apply hierarchical clustering to subsample
hierarchical = AgglomerativeClustering(n_clusters=4, linkage='ward')
hierarchical_labels_sample = hierarchical.fit_predict(X_sample)

# 3. Evaluation
kmeans_silhouette = silhouette_score(
    X_scaled, kmeans_labels, sample_size=1000)  # Subsample for speed
hierarchical_silhouette = silhouette_score(
    X_sample, hierarchical_labels_sample)
dbscan_silhouette = None
if n_clusters_dbscan > 1:
    dbscan_silhouette = silhouette_score(
        X_scaled, dbscan_labels, sample_size=1000)

print(f'Silhouette Scores:')
print(f'K-Means: {kmeans_silhouette:.3f}')
print(f'Hierarchical (subsample): {hierarchical_silhouette:.3f}')
print(f'DBSCAN: {dbscan_silhouette if dbscan_silhouette else "N/A"}')

# Visualize K-Means clusters in PCA space
plt.figure(figsize=(6, 4))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_labels,
            cmap='viridis', alpha=0.3, s=10)
plt.title('K-Means Clusters in PCA Space')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar(label='Cluster')
plt.savefig('kmeans_clusters.png')
plt.close()

# 4. Cluster Interpretation
df['cluster'] = kmeans_labels
cluster_summary = df.groupby(
    'cluster')[['demand', 'temperature', 'humidity', 'hour', 'month']].mean()
print('Cluster Characteristics:')
print(cluster_summary)

for cluster in cluster_summary.index:
    temp = cluster_summary.loc[cluster, 'temperature']
    demand = cluster_summary.loc[cluster, 'demand']
    hour = cluster_summary.loc[cluster, 'hour']
    month = cluster_summary.loc[cluster, 'month']
    print(f'Cluster {cluster}:')
    print(f'- Average Demand: {demand:.0f}')
    print(f'- Average Temperature: {temp:.2f}')
    print(f'- Average Hour: {hour:.1f}')
    print(f'- Average Month: {month:.1f}')
    if temp > 0.7 and hour > 12:
        print('- Likely: High-demand hot afternoons')
    elif temp < 0.3 and hour < 6:
        print('- Likely: Low-demand cool nights')
    else:
        print('- Mixed patterns')
    print()

# Save results
df.to_csv('clustered_data.csv', index=False)

DBSCAN: 5859 clusters, 79528 noise points
Silhouette Scores:
K-Means: 0.138
Hierarchical (subsample): 0.111
DBSCAN: -0.2994783818721771
Cluster Characteristics:
               demand  temperature  humidity       hour     month
cluster                                                          
0         7309.269531     0.470318  0.909456  12.467238  6.960685
1        10837.691406     0.701908  0.516355  14.255876  7.481997
2         6473.314453     0.509914  0.741092   9.600948  9.791533
3         6494.676270     0.458724  0.685714  10.840166  2.657360
Cluster 0:
- Average Demand: 7309
- Average Temperature: 0.47
- Average Hour: 12.5
- Average Month: 7.0
- Mixed patterns

Cluster 1:
- Average Demand: 10838
- Average Temperature: 0.70
- Average Hour: 14.3
- Average Month: 7.5
- Likely: High-demand hot afternoons

Cluster 2:
- Average Demand: 6473
- Average Temperature: 0.51
- Average Hour: 9.6
- Average Month: 9.8
- Mixed patterns

Cluster 3:
- Average Demand: 6495
- Average Temperature: 

Predictive Modeling

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('processed_data.csv')

# Optimize data types to reduce memory
df = df.astype(
    {col: 'float32' for col in df.select_dtypes(include='float64').columns})

# Select features for clustering and forecasting
features = ['demand', 'precipIntensity', 'precipProbability', 'temperature', 'humidity',
            'pressure', 'windSpeed', 'hour', 'day_of_week', 'month', 'year', 'anomaly']
X = df[features]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X).astype(
    np.float32)  # Use float32 to save memory

# 1. Clustering Task
# Dimensionality Reduction
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
explained_variance = pca.explained_variance_ratio_

tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=250)
X_tsne = tsne.fit_transform(X_scaled[:5000])  # Reduced subsample for speed

# Visualize PCA and t-SNE
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.3, s=10)
plt.title(f'PCA ({explained_variance.sum():.2%} Variance)')
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.subplot(1, 2, 2)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], alpha=0.3, s=10)
plt.title('t-SNE')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.tight_layout()
plt.savefig('dimensionality_reduction.png')
plt.close()

# K-Means: Elbow Method
inertia = []
K = range(1, 10)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(6, 4))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.savefig('elbow_curve.png')
plt.close()

optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
dbscan_labels = dbscan.fit_predict(X_scaled)

n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)
print(f'DBSCAN: {n_clusters_dbscan} clusters, {n_noise} noise points')

# Hierarchical Clustering (subsample to avoid memory error)
sample_size = 1000
np.random.seed(42)
sample_indices = np.random.choice(
    X_scaled.shape[0], sample_size, replace=False)
X_sample = X_scaled[sample_indices]

# Dendrogram
Z = linkage(X_sample, method='ward', metric='euclidean')
plt.figure(figsize=(6, 4))
dendrogram(Z, truncate_mode='level', p=5)
plt.title('Dendrogram for Hierarchical Clustering')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.savefig('dendrogram.png')
plt.close()

# Apply hierarchical clustering to subsample
hierarchical = AgglomerativeClustering(n_clusters=4, linkage='ward')
hierarchical_labels_sample = hierarchical.fit_predict(X_sample)

# Clustering Evaluation
kmeans_silhouette = silhouette_score(X_scaled, kmeans_labels, sample_size=1000)
hierarchical_silhouette = silhouette_score(
    X_sample, hierarchical_labels_sample)
dbscan_silhouette = None
if n_clusters_dbscan > 1:
    dbscan_silhouette = silhouette_score(
        X_scaled, dbscan_labels, sample_size=1000)

print(f'Silhouette Scores:')
print(f'K-Means: {kmeans_silhouette:.3f}')
print(f'Hierarchical (subsample): {hierarchical_silhouette:.3f}')
print(f'DBSCAN: {dbscan_silhouette if dbscan_silhouette else "N/A"}')

# Visualize K-Means clusters
plt.figure(figsize=(6, 4))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_labels,
            cmap='viridis', alpha=0.3, s=10)
plt.title('K-Means Clusters in PCA Space')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar(label='Cluster')
plt.savefig('kmeans_clusters.png')
plt.close()

# Cluster Interpretation
df['cluster'] = kmeans_labels
cluster_summary = df.groupby(
    'cluster')[['demand', 'temperature', 'humidity', 'hour', 'month']].mean()
print('Cluster Characteristics:')
print(cluster_summary)

for cluster in cluster_summary.index:
    temp = cluster_summary.loc[cluster, 'temperature']
    demand = cluster_summary.loc[cluster, 'demand']
    hour = cluster_summary.loc[cluster, 'hour']
    month = cluster_summary.loc[cluster, 'month']
    print(f'Cluster {cluster}:')
    print(f'- Average Demand: {demand:.0f}')
    print(f'- Average Temperature: {temp:.2f}')
    print(f'- Average Hour: {hour:.1f}')
    print(f'- Average Month: {month:.1f}')
    if temp > 0.7 and hour > 12:
        print('- Likely: High-demand hot afternoons')
    elif temp < 0.3 and hour < 6:
        print('- Likely: Low-demand cool nights')
    else:
        print('- Mixed patterns')
    print()

# 2. Predictive Modeling
# Prepare data for forecasting
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df = df.sort_values('timestamp')

# Create lag feature for previous day's demand
df['demand_lag_24'] = df['demand'].shift(24)
df = df.dropna()

# Features for forecasting
forecast_features = ['temperature', 'humidity', 'pressure', 'windSpeed', 'hour',
                     'day_of_week', 'month', 'year', 'demand_lag_24']
X_forecast = df[forecast_features].astype(np.float32)
y_forecast = df['demand'].astype(np.float32)

# Split data
train_size = int(len(df) * 0.8)
X_train, X_test = X_forecast[:train_size], X_forecast[train_size:]
y_train, y_test = y_forecast[:train_size], y_forecast[train_size:]

# Baseline: Naive forecast
naive_forecast = df['demand_lag_24'][train_size:].astype(np.float32)
naive_mae = mean_absolute_error(y_test, naive_forecast)
naive_rmse = np.sqrt(mean_squared_error(y_test, naive_forecast))
naive_mape = np.mean(np.abs((y_test - naive_forecast) / y_test)) * 100

print('Naive Forecast Metrics:')
print(f'MAE: {naive_mae:.2f}')
print(f'RMSE: {naive_rmse:.2f}')
print(f'MAPE: {naive_mape:.2f}%')

# Random Forest
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_params = {'n_estimators': [100], 'max_depth': [10]}
rf_grid = GridSearchCV(rf, rf_params, cv=3,
                       scoring='neg_mean_absolute_error', n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_best = rf_grid.best_estimator_
rf_pred = rf_best.predict(X_test)

rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_mape = np.mean(np.abs((y_test - rf_pred) / y_test)) * 100

print('Random Forest Metrics:')
print(f'MAE: {rf_mae:.2f}')
print(f'RMSE: {rf_rmse:.2f}')
print(f'MAPE: {rf_mape:.2f}%')

# Gradient Boosting
gb = GradientBoostingRegressor(random_state=42)
gb_params = {'n_estimators': [100], 'max_depth': [3]}
gb_grid = GridSearchCV(gb, gb_params, cv=3, scoring='neg_mean_absolute_error')
gb_grid.fit(X_train, y_train)
gb_best = gb_grid.best_estimator_
gb_pred = gb_best.predict(X_test)

gb_mae = mean_absolute_error(y_test, gb_pred)
gb_rmse = np.sqrt(mean_squared_error(y_test, gb_pred))
gb_mape = np.mean(np.abs((y_test - gb_pred) / y_test)) * 100

print('Gradient Boosting Metrics:')
print(f'MAE: {gb_mae:.2f}')
print(f'RMSE: {gb_rmse:.2f}')
print(f'MAPE: {gb_mape:.2f}%')

# Ensemble: Stacking
estimators = [('rf', rf_best), ('gb', gb_best)]
stacking = StackingRegressor(
    estimators=estimators, final_estimator=LinearRegression(), n_jobs=-1)
stacking.fit(X_train, y_train)
stack_pred = stacking.predict(X_test)

stack_mae = mean_absolute_error(y_test, stack_pred)
stack_rmse = np.sqrt(mean_squared_error(y_test, stack_pred))
stack_mape = np.mean(np.abs((y_test - stack_pred) / y_test)) * 100

print('Stacking Ensemble Metrics:')
print(f'MAE: {stack_mae:.2f}')
print(f'RMSE: {stack_rmse:.2f}')
print(f'MAPE: {stack_mape:.2f}%')

# Visualize forecasts
plt.figure(figsize=(8, 4))
plt.plot(y_test.values[:168], label='Actual', alpha=0.7)
plt.plot(stack_pred[:168], label='Stacking Predicted', alpha=0.7)
plt.plot(naive_forecast.values[:168], label='Naive', alpha=0.7)
plt.title('Actual vs Predicted Demand (First Week)')
plt.xlabel('Hour')
plt.ylabel('Demand (MWh)')
plt.legend()
plt.savefig('forecast_comparison.png')
plt.close()

# Save results
df.to_csv('clustered_and_forecasted_data.csv', index=False)

DBSCAN: 5859 clusters, 79528 noise points
Silhouette Scores:
K-Means: 0.138
Hierarchical (subsample): 0.111
DBSCAN: -0.2994783818721771
Cluster Characteristics:
               demand  temperature  humidity       hour     month
cluster                                                          
0         7309.269531     0.470318  0.909456  12.467238  6.960685
1        10837.691406     0.701908  0.516355  14.255876  7.481997
2         6473.314453     0.509914  0.741092   9.600948  9.791533
3         6494.676270     0.458724  0.685714  10.840166  2.657360
Cluster 0:
- Average Demand: 7309
- Average Temperature: 0.47
- Average Hour: 12.5
- Average Month: 7.0
- Mixed patterns

Cluster 1:
- Average Demand: 10838
- Average Temperature: 0.70
- Average Hour: 14.3
- Average Month: 7.5
- Likely: High-demand hot afternoons

Cluster 2:
- Average Demand: 6473
- Average Temperature: 0.51
- Average Hour: 9.6
- Average Month: 9.8
- Mixed patterns

Cluster 3:
- Average Demand: 6495
- Average Temperature: 